# Background

- The platform has 6 years worth of data.
- Problem: Issues with retention, reengagement, campaign targeting, and lack of general understanding of the user base
- Ask: How to group players based on recent behavior, general purchase behavior, and extreme purchase behavior

# Data Profile

- Business domain: scheduling platform for services
- Snapshot date: 2022-09-03
- Disclaimer: Data is private and have been masked. It will not be shared.

# Setup

In [1]:
%pip install ../.

Processing c:\users\franc\git-repo\scheduling-platform-segmentation
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scheduling-platform-segmentation: filename=scheduling_platform_segmentation-0.0.1-py3-none-any.whl size=4425 sha256=fb8db1e4d45e74704dcbc13d2cf2ee49c93aa0474b1fdae9df342b2e2166c03c
  Stored in directory: c:\users\franc\appdata\local\pip\cache\wheels\0f\b0\cc\22072402b1c65edb72b7efff9548c7464e42d8420b35ab478b
Successfully built scheduling-platform-segmentation
  Attempting uninstall: scheduling-platform-segmentation
    Found existing installation: scheduling-platform-segmentation 0.0.1
    Uninstalling scheduling-platform-segmentation-0.0.1:
      Successfully uninstalled scheduling-platform-segmentation-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

from copy import deepcopy

from scheduling_platform_segmentation.constants import QUANTILE_LIST, TRAD_RF_SEGMENTS_LIST
from scheduling_platform_segmentation.preprocess.rfm import (
    generate_quantile_dictionary,
    generate_conditions,
    generate_trad_rf_segments
)

import warnings
warnings.filterwarnings("ignore")

In [3]:
pdf_raw_user_by_vendor_by_store = pd.read_csv('user_data_by_vendor_by_store.csv')
pdf_raw_user_by_vendor_by_store = pdf_raw_user_by_vendor_by_store[[c for c in pdf_raw_user_by_vendor_by_store.columns if c != 'Unnamed: 0']]
pdf_raw_user_by_vendor_by_store

,user_id,install_date,vendor_id,store_id,tenure,store_days_since_last_transaction,store_num_transactions,store_total_dollar_spend
0,142488,2022-07-23,195,269,41.737250,12.899776,2,15400.0
1,133793,2022-06-01,195,269,93.859821,20.668556,3,33000.0
2,139901,2022-07-09,195,269,55.806231,55.806223,1,33000.0
3,134030,2022-06-02,195,269,92.660766,63.704679,3,19600.0
4,140978,2022-07-14,195,269,50.750846,23.745670,3,13400.0
...,...,...,...,...,...,...,...,...
92250,110297,2021-11-29,139,219,277.861396,81.845761,2,50300.0
92251,108382,2021-11-15,139,219,291.649244,88.853760,2,59196.0
92252,123325,2022-03-22,139,219,164.875360,95.752685,1,67050.0
92253,17108,2018-08-22,146,226,1472.914944,415.731643,1,500.0


In [4]:
# pdf_raw_user_purchase_behavior = pd.read_csv('user_data_purchase_behavior.csv')
# pdf_raw_user_purchase_behavior = pdf_raw_user_purchase_behavior[[c for c in pdf_raw_user_purchase_behavior.columns if c != 'Unnamed: 0']]
# pdf_raw_user_purchase_behavior

# Recent Purchase Behavior

In [5]:
pdf_raw_user_by_vendor_by_store

,user_id,install_date,vendor_id,store_id,tenure,store_days_since_last_transaction,store_num_transactions,store_total_dollar_spend
0,142488,2022-07-23,195,269,41.737250,12.899776,2,15400.0
1,133793,2022-06-01,195,269,93.859821,20.668556,3,33000.0
2,139901,2022-07-09,195,269,55.806231,55.806223,1,33000.0
3,134030,2022-06-02,195,269,92.660766,63.704679,3,19600.0
4,140978,2022-07-14,195,269,50.750846,23.745670,3,13400.0
...,...,...,...,...,...,...,...,...
92250,110297,2021-11-29,139,219,277.861396,81.845761,2,50300.0
92251,108382,2021-11-15,139,219,291.649244,88.853760,2,59196.0
92252,123325,2022-03-22,139,219,164.875360,95.752685,1,67050.0
92253,17108,2018-08-22,146,226,1472.914944,415.731643,1,500.0


In [6]:
pdf_user_level = (
    pdf_raw_user_by_vendor_by_store
    .groupby('user_id')
    .agg(install_date=('install_date', 'min'),
         cnt_vendors=('vendor_id', 'nunique'),
         tenure=('tenure', 'max'),
         recency=('store_days_since_last_transaction', 'min'),
         frequency=('store_num_transactions', 'sum'),
         monetary=('store_total_dollar_spend', 'sum')
         )
    )



In [7]:
pdf_user_level

,install_date,cnt_vendors,tenure,recency,frequency,monetary
user_id,,,,,,
5,2016-03-31,1,2346.602576,70.874920,3,1495.0
52,2016-07-15,1,2240.230525,1574.828366,1,300.0
56,2016-09-01,6,2192.329370,535.607214,14,7480.0
57,2016-09-08,4,2185.122283,1742.387525,6,4098.0
73,2016-11-11,1,2121.104485,939.375335,2,755.0
...,...,...,...,...,...,...
148312,2022-09-02,1,0.724430,0.724201,1,550.0
148333,2022-09-02,1,0.619158,0.619144,1,3028.0
148347,2022-09-02,1,0.544632,0.544619,1,798.0


In [8]:
dict_quantile = generate_quantile_dictionary(pdf_user_level, QUANTILE_LIST) # save this into database
dict_quantile

{'tenure': {0.2: 307.80049852,
  0.4: 681.7994606200001,
  0.6: 954.70483208,
  0.8: 1273.7553182000001},
 'recency': {0.2: 190.58063782000002,
  0.4: 625.7738795,
  0.6: 795.4346637599999,
  0.8: 1124.701215},
 'frequency': {0.2: 1.0, 0.4: 1.0, 0.6: 1.0, 0.8: 2.0},
 'monetary': {0.2: 230.0, 0.4: 450.0, 0.6: 795.0, 0.8: 1745.0}}

In [9]:
pdf_rfm_scored = deepcopy(pdf_user_level)
for metric in dict_quantile.keys():
    if metric in ('recency'):
        reverse = 1
    else:
        reverse = 0

    conditions, values = generate_conditions(pdf_rfm_scored, dict_quantile, metric, reverse=reverse)
    pdf_rfm_scored[metric[0]] = np.select(conditions, values)

pdf_rfm_scored

,install_date,cnt_vendors,tenure,recency,frequency,monetary,t,r,f,m
user_id,,,,,,,,,,
5,2016-03-31,1,2346.602576,70.874920,3,1495.0,5,5,5,4
52,2016-07-15,1,2240.230525,1574.828366,1,300.0,5,1,1,2
56,2016-09-01,6,2192.329370,535.607214,14,7480.0,5,4,5,5
57,2016-09-08,4,2185.122283,1742.387525,6,4098.0,5,1,5,5
73,2016-11-11,1,2121.104485,939.375335,2,755.0,5,2,4,3
...,...,...,...,...,...,...,...,...,...,...
148312,2022-09-02,1,0.724430,0.724201,1,550.0,1,5,1,3
148333,2022-09-02,1,0.619158,0.619144,1,3028.0,1,5,1,5
148347,2022-09-02,1,0.544632,0.544619,1,798.0,1,5,1,4


In [10]:
pdf_rfm_labeled = generate_trad_rf_segments(pdf_rfm_scored)
pdf_rfm_labeled

,install_date,cnt_vendors,tenure,recency,frequency,monetary,t,r,f,m,wtd_rfm
106246,2021-10-30,1,307.799153,121.934572,6,90396.0,1,5,5,5,champions
106247,2021-10-30,1,307.797483,139.782477,1,3826.0,1,5,1,5,new_customers
106251,2021-10-30,1,307.784611,307.784600,1,42800.0,1,4,1,5,promising
106252,2021-10-30,1,307.776173,223.626908,2,1495.0,1,4,4,4,loyal_customers
106254,2021-10-30,1,307.769778,117.760101,2,1496.0,1,5,4,4,champions
...,...,...,...,...,...,...,...,...,...,...,...
34853,2019-03-09,1,1273.808965,985.000000,4,1745.0,5,2,5,4,cant_lose
34854,2019-03-09,1,1273.799046,1080.489532,2,790.0,5,2,4,3,at_risk
34855,2019-03-09,1,1273.781311,1147.726193,1,450.0,5,1,1,2,hibernating
34856,2019-03-09,1,1273.769372,1084.455102,2,845.0,5,2,4,4,at_risk
